In [1]:
import cv2
import cupy as np
from os import listdir
from sandbox import activations, costs, layers, predictions, model

In [2]:
# Load altered pizza dataset
dataset_path = 'dataset\\altered\\'
datasets = [
    np.array([
        cv2.imread(dataset_path + dir + img).flatten()
        for img in listdir(dataset_path + dir)
    ])
    for dir in ['train\\pizza\\', 'train\\not_pizza\\', 'test\\pizza\\', 'test\\not_pizza\\']
]

# Create training and testing data
train_x = np.concatenate((datasets[0], datasets[1]), axis=0) / 255
train_y = np.concatenate((np.ones((datasets[0].shape[0], 1)), np.zeros((datasets[1].shape[0], 1))), axis=0)

test_x = np.concatenate((datasets[2], datasets[3]), axis=0) / 255
test_y = np.concatenate((np.ones((datasets[2].shape[0], 1)), np.zeros((datasets[3].shape[0], 1))), axis=0)

In [3]:
# Create model
pizza = model.Model(cuda=True)
pizza.add(layers.Dense(units=32, activation=activations.ReLU()))
pizza.add(layers.Dense(units=16, activation=activations.ReLU()))
pizza.add(layers.Dense(units=8, activation=activations.ReLU()))
pizza.add(layers.Dense(units=1, activation=activations.Sigmoid()))
pizza.configure(learning_rate=0.01, epochs=2500, cost_type=costs.BinaryCrossentropy())

# Train model
pizza.train(train_x, train_y, verbose=True)

Cost on epoch 0: 0.70747
Cost on epoch 100: 0.65237
Cost on epoch 200: 0.63867
Cost on epoch 300: 0.62769
Cost on epoch 400: 0.61934
Cost on epoch 500: 0.6128
Cost on epoch 600: 0.60599
Cost on epoch 700: 0.59919
Cost on epoch 800: 0.59212
Cost on epoch 900: 0.58475
Cost on epoch 1000: 0.57887
Cost on epoch 1100: 0.57311
Cost on epoch 1200: 0.5666
Cost on epoch 1300: 0.56142
Cost on epoch 1400: 0.55693
Cost on epoch 1500: 0.55357
Cost on epoch 1600: 0.547
Cost on epoch 1700: 0.54353
Cost on epoch 1800: 0.53919
Cost on epoch 1900: 0.53522
Cost on epoch 2000: 0.52829
Cost on epoch 2100: 0.52488
Cost on epoch 2200: 0.521
Cost on epoch 2300: 0.51507
Cost on epoch 2400: 0.51234
Cost on epoch 2500: 0.51034


In [4]:
# Assess model accuracy
pred_train = pizza.predict(train_x, prediction_type=predictions.binary_classification) # Get model accuracy on training data
print('\nTraining Accuracy: ' + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = pizza.predict(test_x, prediction_type=predictions.binary_classification) # Get model accuracy on testing data
print('Testing Accuracy: ' + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))


Training Accuracy: 0.77301
Testing Accuracy: 0.71217
